# Data exploration
This notebook is used to navigate the different CSV file and understand how the data is structured and find whether entries in a column are unique or repeated. This is used to determine which fields can be used as primary keys and for which tables composite keys may be required. It is also used to determine the appropriate datatypes.

**NOTE**: while this code is useful to automatically extract information about the data, the head of the DataFrame are displayed to allow the user to visually check the data and make educated decisions. This is especially useful when it comes to date that this code does not yet handled automatically.

In [2]:
# Dependencies
import pandas as pd
from pathlib import Path
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

## Exploration function
The function below is used to analyse the data in the files. It looks at the length of the file (number of rows), the number of columns and whether each column can be used as a primary key by comparing the number of unique values to the number or rows in the file. It also determine whether the values are numerical or strings. In case of strings, it looks at the maximum number of character to determine what the maximum VARCHAR length should be in the database.

The function return a DataFrame containing the CSV file data for further analysis.

In [6]:
def explore_data(filename):
    # Print file name
    print(f"File: {filename}")

    # Load the CSV file into a DataFrame
    csvfile = Path(f'{filename}')
    df = pd.read_csv(csvfile)

    # Size of file
    print(f"Length of file: {len(df)} rows")

    # Number of columns
    num_col = len(df.columns)
    print(f"Number of columns: {num_col}")

    for i in range(num_col):
        # Count the number of unique values in column i
        output_string = f"- Column {i}: '{df.columns[i]}' has {df[df.columns[i]].nunique()} unique values"
    
        # If the number of unique value in column i is the same as the number of rows...
        if df[df.columns[i]].nunique() == len(df):
            # ... then the column can be used as a primary key
            output_string += ', and can be used as primary key'
    
        # Check whether the data is numeric
        if is_numeric_dtype(df[df.columns[i]]):
            # The code will suggest to use INT for any numeric value. This must be checked by the user.
            output_string += '. Values are numeric. Suggest: INT'
        
        # Check whether the data are characters
        elif is_string_dtype(df[df.columns[i]]):

            max_length = df[df.columns[i]].map(lambda x: len(x)).max()

            if max_length == 1:
                # If there is only one charachter, CHAR(1) is suggested as data type.
                output_string += f". Values are strings. Suggest: CHAR(1)"
            else:
                # If the maximum number of character is greater than one, VARCHAR(max) is suggested, a larger number may be used.
                output_string += f". Values are strings. Suggest: VARCHAR({max_length})"

        # State that the datatype cannot be determine
        else:
            output_string += '. Values are of an unknown type.'

        # Print the information about column i
        print(output_string)

    print('Extract of data:')
    display(df.head(2))
    return df

## File 1: category.csv

In [7]:
filename = 'Resources/category.csv'
df = explore_data(filename)

File: Resources/category.csv
Length of file: 9 rows
Number of columns: 2
- Column 0: 'category_id' has 9 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(4)
- Column 1: 'category' has 9 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(12)
Extract of data:


,category_id,category
0,cat1,food
1,cat2,music


## File 2: subcategory.csv

In [8]:
filename = 'Resources/subcategory.csv'
df = explore_data(filename)

File: Resources/subcategory.csv
Length of file: 24 rows
Number of columns: 2
- Column 0: 'subcategory_id' has 24 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(8)
- Column 1: 'subcategory' has 24 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(17)
Extract of data:


,subcategory_id,subcategory
0,subcat1,food trucks
1,subcat2,rock


## File 3: campaign.csv

In [9]:
filename = 'Resources/campaign_BChamot.csv'
df = explore_data(filename)

File: Resources/campaign_BChamot.csv
Length of file: 1000 rows
Number of columns: 14
- Column 0: 'cf_id' has 1000 unique values, and can be used as primary key. Values are numeric. Suggest: INT
- Column 1: 'contact_id' has 1000 unique values, and can be used as primary key. Values are numeric. Suggest: INT
- Column 2: 'company_name' has 974 unique values. Values are strings. Suggest: VARCHAR(33)
- Column 3: 'description' has 999 unique values. Values are strings. Suggest: VARCHAR(53)
- Column 4: 'goal' has 447 unique values. Values are numeric. Suggest: INT
- Column 5: 'pledged' has 964 unique values. Values are numeric. Suggest: INT
- Column 6: 'outcome' has 4 unique values. Values are strings. Suggest: VARCHAR(10)
- Column 7: 'backers_count' has 589 unique values. Values are numeric. Suggest: INT
- Column 8: 'country' has 7 unique values. Values are strings. Suggest: VARCHAR(2)
- Column 9: 'currency' has 7 unique values. Values are strings. Suggest: VARCHAR(3)
- Column 10: 'launched_

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1
1,1175,2288,Werner-Bryant,Virtual uniform frame,1800.0,7991.0,successful,222,US,USD,2020-06-20,2021-01-30,cat1,subcat1


## File 4: contacts.csv

In [10]:
filename = 'Resources/contacts_BChamot.csv'
df = explore_data(filename)

File: Resources/contacts_BChamot.csv
Length of file: 1000 rows
Number of columns: 4
- Column 0: 'contact_id' has 1000 unique values, and can be used as primary key. Values are numeric. Suggest: INT
- Column 1: 'first_name' has 1000 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(12)
- Column 2: 'last_name' has 1000 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(13)
- Column 3: 'email' has 1000 unique values, and can be used as primary key. Values are strings. Suggest: VARCHAR(42)
Extract of data:


,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org


# Conclusions from the exploration (OLD DATA)
## Conclusions for Table 1: departments
1. Column `dept_no` should be used as PRIMARY KEY.
2. No FOREIGN KEY are used.

## Conclusion for Table 2: dept_emp
1. Column `emp_no` has repeating values. An employee can therefore be part of multiple departments.
2. Column `dept_no` has repeating values. A depeartment can therefore include multiple employees.
3. No PRIMARY KEY can be defined with the existing values.

## Conclusions for Table 3: dept_manager
1. Column `dept_no` has repeating values. A department may therefore have multiple managers.
2. Column `emp_no` has no repeating values. An employee can then be manager in only one department.

## Conclusions for Table 4: employees
1. Column `emp_no` should be used as PRIMARY KEY.
2. All the other columns have repeating values.
3. Columns `birth_date` and `hire_date` are in format MDY (US)

# Quick Database Diagram (OLD DATA)
The following code can be used on https://app.quickdatabasediagrams.com to create the data model.